# 🚂 Sistema Ollama + FastAPI
## Modelos de IA para Análise de Transporte Ferroviário

Este notebook permite usar modelos de linguagem local para análises de transporte:
- **TinyLLama**: Modelo rápido e leve (1.1B parâmetros)
- **Qwen2:1.5b**: Modelo mais preciso (1.5B parâmetros) - pode ser lento

### ⚡ Como usar:
1. **IMPORTANTE**: Certifique-se de que o Docker esteja rodando: `docker compose up -d`
2. Execute a **Célula 1**: Configuração inicial
3. Execute a **Célula 2**: Verificar modelos disponíveis  
4. Execute a **Célula 3**: Teste rápido
5. Use as **Células 4-5**: Para consultas específicas

### ⚠️ Notas importantes:
- Os modelos podem demorar 30-60s para responder
- Se der timeout, tente novamente ou use o TinyLLama
- Primeiro download dos modelos pode demorar vários minutos

In [ ]:
# 1. CONFIGURAÇÃO INICIAL
from chat_client import ChatClient
import time

# Configurar cliente
client = ChatClient(base_url="http://localhost:8000")

print("✅ Cliente configurado")
print("✅ URL da API:", client.base_url)

# Verificar status da API
try:
    health = client.health_check()
    print("✅ API funcionando:", health)
except:
    print("❌ API não está respondendo. Execute: docker compose up -d")

MODELOS DE BAIXO CONSUMO

"tinyllama:latest" </br>
"qwen2:1.5b"</br>
"deepcoder:1.5b"</br>
"stablelm2:1.6b"</br>
"tinydolphin:latest"</br>
"exaone-deep:2.4b"</br>

</br>
granite3.3:2b</br>
granite3.1-dense:2b</br>

In [ ]:
modelos_a_baixar = ["tinyllama:latest","qwen2:1.5b","qwen3:1.7b"]

In [ ]:
# 2. VERIFICAR MODELOS DISPONÍVEIS
print("Verificando modelos disponíveis...")

# Listar modelos já instalados
modelos_disponiveis = client.listar_modelos()
print(f"Modelos instalados: {modelos_disponiveis}")

# Verificar se temos os modelos necessários
for modelo in modelos_a_baixar:
    if modelo in modelos_disponiveis:
        print(f"✓ {modelo}: disponível")
    else:
        print(f"✗ {modelo}: não encontrado, baixando")
        client.baixar_modelo(modelo)

print("\nPronto para usar os modelos disponíveis!")

In [ ]:
contexto = """
Plano Estratégico Ferroviário de Minas Gerais
Perfil da Proposta da Ferrovia para Transporte de Passageiros
Proposta: São Sebastião do Rio Verde / Passa Quatro Código: TUR 59
Categoria: Proposta Turística Versão: 00
Tipo de empreendimento:
Greenfield
Brownfield (existência da faixa de domínio,
com reconstrução da via permanente)
Características físicas:
Extensão (km): 25,28 Tipo bitola: -
Total de estações: 3
Estações atendidas (extensão acumulado em km):
São Sebastião do Rio Verde 0,00
Itanhandu 13,43
Passa Quatro 25,28
Características operacionais:
Tempo de viagem ida (min): 76,2 Tempo de viagem ida & volta (min): 152,4
Viagens (mês): 5 (ida + volta) Dias de operação (mês): 5
Demanda (mês): 1.924 Produção quilométrica (km/mês): 252,8
Tarifa do serviço: 65,00 Tarifa ida e volta (R$)
Desempenho da linha:
Receita anual (R$): 1 500 720,00
Pass.ano/km: 7,6 Receita.ano/km: 494,7
"""

In [ ]:
# 5. SUA CONSULTA PERSONALIZADA
# Modifique a pergunta abaixo e execute para suas proprias analises

# Escolha o modelo (tinyllama:latest é mais rapido e confiavel)
modelo = "tinyllama:latest"
modelo = "tinydolphin:latest"  
modelo = "qwen2:1.5b"  # Modelo mais confiavel
# modelo = "deepcoder:1.5b"     # Erro de resposta, timeout
# modelo = "exaone-deep:2.4b"   # Erro de resposta, timeout
# modelo = "stablelm2:1.6b"     # Resultado 100% errado
modelo = "qwen3:1.7b"         # Erro de resposta, timeout

# Sua pergunta personalizada - MODIFIQUE AQUI:
# os dados sempres estarão assim:  <key>: <value>

print(f"Modelo selecionado: {modelo}")
print("\nProcessando...")

# Adicionar contexto para melhorar resposta
prompt = f"""
você foi designado a responder o formulario <json> com base nos dados no <contexto>,
então responda rapidamente e com precisão.

# Formulario JSON:
<json>
{{
    "Proposta": value1,
    "Código": value2,
    "Categoria": value3,
    "Tipo de empreendimento": ["empreendimento1", "empreendimento2", "empreendimento3" ...], # RESPONDA COM OS NOMES DOS EMPREENDIMENTOS

    "Extensão (km)": value5,
    "Tipo bitola": value6,
    "Total de estações": value7,
    "Estações atendidas": ["estação1", "estação2", "estação3" ...], # RESPONDA COM OS NOMES DE ESTAÇÕES

    "Tempo de viagem ida (min)": value9,
    "Tempo de viagem ida & volta (min)": value10,
    "Viagens (mês)": value11,
    "Dias de operação (mês)": value12,
    "Demanda (mês)": value13,
    "Produção quilométrica (km/mês)": value14,
    "Tarifa do serviço": value15,

    "Receita anual (R$)": value16,
    "Pass.ano/km": value17,
    "Receita.ano/km": value18
}}
</json>

# Fonte de dados:
<contexto> 
{contexto}
</contexto> 

<tarefa>
extraia os dados da fonde dados com a tag <contexto> e preencha o json com tag <json> nos campos value1, value2, etc.
</tarefa>

<resposta>
retorne o formulario JSON preenchido, e quando falar dele inicie com a tag <json> e quando finalizar insira a tag </json>.
</resposta>
"""

try:
    inicio = time.time()
    resposta = client.chat(prompt, modelo=modelo, stream=False, timeout=6000)
    tempo = time.time() - inicio

    print(f"\nTempo: {tempo:.1f}s")
    print(f"\nResposta Especializada:")
    print("=" * 60)
    if 'resposta' in resposta:
        print(resposta['resposta'])
    elif 'erro' in resposta:
        print(f"Erro: {resposta['erro']}")
    else:
        print(str(resposta))
    print("=" * 60)
    
    print("\nPara fazer outra pergunta:")
    print("1. Modifique a variavel 'sua_pergunta' acima")
    print("2. Execute esta celula novamente")
except Exception as e:
    print(f"Erro: {str(e)}")
    print("Verifique se o Docker está rodando: docker compose up -d")

In [ ]:
client.baixar_modelo("qwen3:1.7b")

In [ ]:

modelo
